In [1]:
import pandas as pd
import numpy as np
from scipy import stats


In [2]:
df1=pd.read_excel('dataset_780.xlsx',sheet_name=0)
df2=pd.read_excel('dataset_430.xlsx')
print (df1['Management'].value_counts())

# confidence = 0.95
# data=df1['Age'].dropna()
# # Calculate mean and standard error
# mean = np.mean(data)
# sem = stats.sem(data)  # Standard Error of the Mean

# # t-distribution-based confidence interval
# ci = stats.t.interval(confidence, df=len(data)-1, loc=mean, scale=sem)

# print(f"Mean: {mean}")
# print(f"95% Confidence Interval: {ci}")
# print (len(df1.columns))



Management
conservative                 483
primary surgical             270
secondary surgical            27
simultaneous appendectomy      1
Name: count, dtype: int64


In [3]:
df1_col=['Sex','Nausea','Loss_of_Appetite','Body_Temperature','Migratory_Pain','Ipsilateral_Rebound_Tenderness','Peritonitis','WBC_Count','Neutrophil_Percentage','Ketones_in_Urine','CRP','Free_Fluids','Appendix_Diameter','Alvarado_Score','Paedriatic_Appendicitis_Score','Diagnosis','Severity']

df2_col=['Sex','Nausea','AppetiteLoss','BodyTemp','MigratoryPain','ReboundTenderness','Peritonitis','WBCCount','NeutrophilPerc','KetonesInUrine','CRPEntry','FreeFluids','AppendixDiameter','AlvaradoScore','PediatricAppendicitisScore','DiagnosisByCriteria','AppendicitisComplications']

mapping={
    'no':0,
    'yes':1,
    'male':1,
    'female':0,
    '+':1,
    '++':2,
    '+++':3,
    'local':1,
    'generalized':2,
    'generalised':2,
    'appendicitis':1,
    'no appendicitis':0,
    'noAppendicitis':0,
    'uncomplicated':0,
    'complicated':1,
}

In [4]:

df1 = df1.dropna(subset=['Diagnosis'])
df1=df1[df1_col].fillna(0)
df2=df2[df2_col].fillna(0)

df2.columns=df1_col

for col in df1_col:
    df1[col]=df1[col].replace(mapping)
    df2[col]=df2[col].replace(mapping)

print(df1.shape)
print(df2.shape)

df_combined = pd.concat([df1, df2], ignore_index=True)
df_combined=df_combined.drop_duplicates()

print(df_combined.shape)

df_combined.to_excel('dataset_combined.xlsx', index=False)

set_df1 = set(map(tuple, df1.to_numpy()))
set_df2 = set(map(tuple, df2.to_numpy()))

diff_set = set_df2 - set_df1

df2_only = pd.DataFrame(list(diff_set), columns=df2.columns)

print(df2_only.shape)

df2_only.to_excel('dataset_unique.xlsx',index=False)


(780, 17)
(430, 17)
(1015, 17)


C:\Users\ajung\AppData\Local\Temp\ipykernel_12268\3777711301.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1[col]=df1[col].replace(mapping)
C:\Users\ajung\AppData\Local\Temp\ipykernel_12268\3777711301.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2[col]=df2[col].replace(mapping)


(236, 17)


In [5]:
tools = ['Alvarado_Score','Paedriatic_Appendicitis_Score','Appendix_Diameter','Diagnosis']
print(df2_only.shape)

df2_usg = df2_only[df2_only['Appendix_Diameter'] != 0]

(236, 17)


In [ ]:
# Performance of tools in unique dataset

df_unique=df2_only[tools]

alvarado_bins=[0,4,6,10] # right = true
alvarado_labels=['<=4','5-6','>=7']

df_unique['as_labels'] = pd.cut(df_unique['Alvarado_Score'],bins=alvarado_bins,labels=alvarado_labels,right=True )

as_contingency_table = pd.crosstab(df_unique['as_labels'], df_unique['Diagnosis'])
print('For Alvarado Score')
print(as_contingency_table)

pas_bins=[0,5,10] # right = true
pas_labels=['<=5','>=6']

df_unique['pas_labels'] = pd.cut(df_unique['Paedriatic_Appendicitis_Score'],bins=pas_bins,labels=pas_labels,right=True )

pas_contingency_table = pd.crosstab(df_unique['pas_labels'], df_unique['Diagnosis'])
print('For PAS')
print(pas_contingency_table)

# For USG

usg_bins=[0,6,50] # right = false
usg_labels=['<6','>=6']

df2_usg['usg_labels'] = pd.cut(df2_usg['Appendix_Diameter'],bins=usg_bins,labels=usg_labels,right=False )

usg_contingency_table = pd.crosstab(df2_usg['usg_labels'], df2_usg['Diagnosis'])
print(usg_contingency_table)
